In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key


In [3]:
#Import weather data

weather_df = pd.read_csv("..\\WeatherPy\\weather_data.csv")
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Busselton,-33.65,115.33,52.79,52,0,12.15,Southern
1,Cidreira,-30.18,-50.21,66.56,86,100,19.62,Southern
2,Yellowknife,62.46,-114.35,23.00,85,90,9.17,Northern
3,Ushuaia,-54.80,-68.30,48.20,61,75,21.92,Southern
4,Okakarara,-20.58,17.43,94.57,10,45,11.63,Southern


In [4]:
#Store variables to be used in map

locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [9]:
#Create map
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
humidity_heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(humidity_heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
#Narrow down dataframe

vacation_df = weather_df.loc[(weather_df["Temperature"] < 85) & (weather_df["Temperature"] > 70) & (weather_df["Wind_Speed"] < 10) & (weather_df["Cloudiness"] < 10)]
vacation_df = vacation_df.dropna()
vacation_df.count()

City           30
Latitude       30
Longitude      30
Temperature    30
Humidity       30
Cloudiness     30
Wind_Speed     30
Hemisphere     30
dtype: int64

In [20]:
hotel_df = pd.DataFrame({"Hotel Name": []})
hotel_df

,Hotel Name


In [50]:
#Work with Google APIs

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": coords,
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

hotels_list = []
for index, row in vacation_df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    response = requests.get(base_url, params)
    hotels = response.json()
    hotels_list.append(hotels["results"][0]["name"])

hotels_list

['Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul et Virginie Hotel & Spa',
 'Veranda Paul e

In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]